# Google Api places
- This notebook has a walktrough of how to extract information from the Api of google places

- it was very difficult to deal with that APi because did not have pagination therefore the way to work was creating a "grid" 
with a very specific distance between the nodes so in that way we can get as much information as we can from each jump from one node to the other

- The data use for the RAG is not going to be the one got from Google Api places, though, was a really good excersise and the code is going to be used as @tool for the Multi-Agent RAG

In [ ]:
import pandas as pd
import requests
from dotenv import load_dotenv,dotenv_values
from copy import deepcopy as dc
import os
import folium
API = dotenv_values('GOOGLE_API_KEY')
import time
import numpy as np

In [ ]:
pd.reset_option("max_columns")
pd.set_option("max_colwidth",None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
start_lat = 40.417113
start_lng = -3.697292

lat_diff = 0.0050
lng_diff = 0.0050


lat_range = np.arange(start_lat - 0.060, start_lat + 0.060, lat_diff) 
lng_range = np.arange(start_lng - 0.060, start_lng + 0.060, lng_diff)  
coordinates = [(lat, lng) for lat in lat_range for lng in lng_range]

for coord in coordinates[:10]:
    print(coord)
print(f"Total coordinate points generated: {len(coordinates)}")

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    r = 6371000  
    return r * c

lat1, lon1 = coordinates[0][0], coordinates[0][1]  
lat2, lon2 = coordinates[1][0], coordinates[1][1]  

distance = haversine(lat1, lon1, lat2, lon2)
print(f"The distance is {distance:.2f} meters")

In [ ]:
map_madrid = folium.Map(location=[40.4168, -3.7038], zoom_start=14)

for (index, row) in coordinates.iterrows():
    folium.Marker(location = [row.loc["lat"], row.loc["long"]]).add_to(map_madrid)

map_madrid


In [ ]:
def trial_function(coords:pd.DataFrame):
    df = []
    round=1
    for lat, lng in coords:
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=300&type=restaurant&type=bakery&type=bar&type=cafe&type=meal_delivery&type=meal_takeaway&key={os.getenv('GOOGLE_API_KEY')}"
        params = {'pagetoken': ''}
        while True:
            response = requests.get(url, params)
            if response.status_code != 200:
                print("There is a problem, please confirm the requests")
                break
            data = response.json()
            df.extend(data.get('results', []))
            next_page_token = data.get('next_page_token')
            if not next_page_token:
                break
            params['pagetoken'] = next_page_token
            
            time.sleep(2)
        time.sleep(4)
        print(f'round {round}')
        round += 1

    return df

data = trial_function(coor)
df = pd.json_normalize(data)
print(len(df))